In [1]:
SCENARIOS = [
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.4.0\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a2\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a3\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a3-new-environment\baseline"
]

In [2]:
import cea.inputlocator
locators = [cea.inputlocator.InputLocator(s) for s in SCENARIOS]
locator = locators[0]

In [3]:
import cea.schemas
schemas = cea.schemas.schemas(plugins=[])

In [7]:
import yaml
schemas_yml = r"C:\Users\darthoma\Documents\GitHub\CityEnergyAnalyst\cea\schemas.yml"
with open(schemas_yml, "r") as fp:
    schemas = yaml.safe_load(fp)

In [12]:
# figure out input locators to use...
# schemas_yml = r"C:\Users\darthoma\Documents\GitHub\CityEnergyAnalyst\cea\schemas.yml"
# with open(schemas_yml, "r") as fp:
#    schemas = yaml.safe_load(fp)

# these are the files we're going to check
default_args = {
    "building": "B1000",
    "network_type": "DH",
    "panel_type": "ET",
    "use": "RESTAURANT",
    "format": "csv"
}

for lm in schemas.keys():
    if not schemas[lm]["file_type"] in ("csv", "shp", "dbf"):
        # ignore excel files etc. for now...
        continue    
    sio = getattr(locator, lm)
    method = sio.original_function
    
    try:
        args = {p: default_args[p] for p in schemas[lm]["parameters"]}
    except KeyError:
        print(f"Failed on lm={lm}")
        raise
        
    try:
        sio.read(**args)
    except FileNotFoundError:
        print(f"Failed on lm={lm}")
        raise

C:\Users\darthoma\Miniconda2\envs\cea\lib\site-packages\cea\schemas.py:249: UserWarning: Dataframe does not conform to schemas.yml specification for get_demand_results_file(missing: set(), extra: {'Ev_kWh', 'GRID_v_kWh'}
  warnings.warn("Dataframe does not conform to schemas.yml specification for {lm}"


FileNotFoundError: [Errno 2] File C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.4.0\baseline\outputs\data\emissions\Total_LCA_mobility.csv does not exist: 'C:\\Users\\darthoma\\Documents\\CEA-Projects\\reference-case-open.3.4.0\\baseline\\outputs\\data\\emissions\\Total_LCA_mobility.csv'

# check these more thoroughly:

- get_optimization_substations_results_file